In [1]:
college_admission_context = """

SASTRA University offers various undergraduate, postgraduate, and doctoral programs in Engineering, Science, Management, Law, and Arts. 
Admissions for B.Tech programs are based on JEE Main scores and +2 marks. The application process opens in April and closes by the end of May. 
There is no entrance exam conducted by SASTRA University for B.Tech. Admission is based on JEE Main and board exam marks. The merit list is prepared with a 50:50 weightage for JEE Main and board marks.

For non-B.Tech programs, admissions are based on academic performance and personal interviews. Applications can be submitted online through the university's portal. 

Fees for B.Tech programs range from ₹1,20,000 to ₹1,80,000 per year, depending on the branch. Scholarships are available based on merit and need.

Hostel accommodation is available with separate facilities for boys and girls. First-year students are given preference. 

For international students, a separate admission cell handles queries. Students must have equivalent qualifications and submit their transcripts.

For more information, applicants can visit www.sastra.edu or contact the admission helpline at 1800-425-44455.
"""


In [2]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch

In [3]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [4]:
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [6]:
context = college_admission_context

In [7]:
def faq_bot(question):
    input_ids = tokenizer.encode(question, context)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_id = input_ids.index(tokenizer.sep_token_id)
    num_a = sep_id+1
    num_b = len(input_ids) - num_a
    segment_ids = [0]*num_a + [1]*num_b
    output = model(torch.tensor([input_ids]), token_type_ids = torch.tensor([segment_ids]))
    answer_st = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if(answer_end >= answer_st):
        answer = ' '.join(tokens[answer_st:answer_end+1])
    else:
         print("I don't know how to answer this question, can you ask another one?")
    final_answer = ""
    for word in answer.split():
        if(word[:2] == '##'):
            final_answer += word[2:]
        else:
            final_answer += ' '+ word
    print(final_answer.strip())

In [8]:
faq_bot("How are B.Tech admissions done at SASTRA University?")

based on jee main scores and + 2 marks


In [9]:
faq_bot(" Are there any scholarships available?")

scholarships are available based on merit and need


In [10]:
faq_bot("How can I apply for admission to SASTRA University?")

online through the university ' s portal


In [11]:
faq_bot("What programs does SASTRA University offer?")

undergraduate , postgraduate , and doctoral


In [12]:
faq_bot("Where can I get more information about the admissions?")

www . sastra . edu
